# Webcam Live on Jupyter over RTSP

This example shows you how to capture a live webcam stream over network using Real Time Streaming Protocol (RTSP), and playback on Jupyter lab.

It is required to attach a webcam to your PC before running the following code.

Environment Setup:
```
- PC     : Ubuntu 22.04 | 192.168.2.x (x: any)
- PYNQ-ZU: PYNQ-3.0     | 192.168.2.99
```

### On PC, install packages:
```shell
sudo apt-get install libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libgstreamer-plugins-bad1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav gstreamer1.0-tools gstreamer1.0-x gstreamer1.0-alsa gstreamer1.0-gl gstreamer1.0-gtk3 gstreamer1.0-qt5 gstreamer1.0-pulseaudio
```

#### If your PC runs Ubuntu-22.04
```
# fix dependencies
sudo apt-get install libgstreamer-plugins-bad1.0-0=1.20.1-1ubuntu2
sudo apt-get install libgstreamer-plugins-bad1.0-dev

# install plugins
sudo apt-get install gstreamer1.0-libav
sudo apt-get install gstreamer1.0-plugins-ugly gstreamer1.0-plugins-bad gstreamer1.0-plugins-good
sudo apt install ffmpeg
```

Attach a webcam to your PC, properly check the existence of the device.
```
ls /dev/video*
```

Assume that the webcam is connected as `/dev/video0`.

On your PC, start streaming the video stream to `192.168.2.99:5600` endpoint by:
```shell
gst-launch-1.0 -v v4l2src device=/dev/video0 ! videoconvert ! video/x-raw,format=I420,width=640,height=480 ! x264enc tune=zerolatency bitrate=480 speed-preset=ultrafast! video/x-h264,profile=baseline ! rtph264pay ! udpsink host=192.168.2.99 port=5600
```

### On PYNQ-ZU, install packages:
```shell
sudo apt install linux-headers-generic dctrl-tools dkms

sudo apt-get install libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libgstreamer-plugins-bad1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav gstreamer1.0-tools gstreamer1.0-x gstreamer1.0-alsa gstreamer1.0-gl gstreamer1.0-gtk3 gstreamer1.0-qt5 gstreamer1.0-pulseaudio
```

User should take a look in the OpenCV-Python's build information to make sure Gstreamer is available on PYNQ-ZU or not.

In [ ]:
import cv2
print(cv2.getBuildInformation())

The information should contain:
    
```
...
  Video I/O:
    DC1394:                      YES (2.2.6)
    FFMPEG:                      YES
      avcodec:                   YES (58.134.100)
      avformat:                  YES (58.76.100)
      avutil:                    YES (56.70.100)
      swscale:                   YES (5.9.100)
      avresample:                NO
    GStreamer:                   YES (1.19.90)
    PvAPI:                       NO
    v4l/v4l2:                    YES (linux/videodev2.h)
    gPhoto2:                     YES
...
```

In [ ]:
import numpy as np
import cv2
from IPython import display
from PIL import Image
import matplotlib.pyplot as py

import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

%matplotlib inline

## Normal live stream playback

In [ ]:
def showVideo(port=5600, scale=0.5):
    
    # remember to leave a space character in the end of each element
    element_1 = "udpsrc port={} ".format(port)
    element_2 = "! application/x-rtp,encoding-name=H264,payload=96 "
    element_3 = "! rtph264depay ! h264parse ! avdec_h264 "
    element_4 = "! videoconvert ! appsink "
    
    gst_pipeline = element_1 + element_2 + element_3 + element_4
    
    video = cv2.VideoCapture(gst_pipeline, cv2.CAP_GSTREAMER)
    
    try:        
        counter = 0
        while(True):
            ret, frame = video.read()
            if not ret:
                counter += 1
                if counter == 100:
                    print("Video stream is not available.")
                    break
                continue

            resized = cv2.resize(frame, (0, 0), fx = scale, fy = scale)
            _,ret_array = cv2.imencode('.jpg', resized) #this fixed the pipeline issue as the imshow was converting to png.
            i = display.Image(data=ret_array)
            display.display(i)
            display.clear_output(wait=True)

    except KeyboardInterrupt:
        print("Released Video Resource from KeyboardInterrupt.")
    finally:
        del video


In [ ]:
showVideo(5600, 0.5)

## Live stream with filters

In [ ]:
def showVideoWithFilter(port=5600, scale=0.5, filter_name=None):
    # remember to leave a space character in the end of each element
    element_1 = "udpsrc port={} ".format(port)
    element_2 = "! application/x-rtp,encoding-name=H264,payload=96 "
    element_3 = "! rtph264depay ! h264parse ! avdec_h264 "
    element_4 = "! videoconvert ! appsink "
    
    gst_pipeline = element_1 + element_2 + element_3 + element_4
    
    video = cv2.VideoCapture(gst_pipeline, cv2.CAP_GSTREAMER)
    
    try:        
        counter = 0
        while(True):
            ret, frame = video.read()
            if not ret:
                counter += 1
                if counter == 100:
                    print("Video stream is not available.")
                    break
                continue

            resized = cv2.resize(frame, (0, 0), fx = scale, fy = scale)
            filtered = resized
            
            ## Filtering
            if filter_name == None:
                pass
            elif filter_name == 'canny':
                filtered = cv2.Canny(resized, 100, 110)
            elif filter_name == 'blur':
                filtered = cv2.GaussianBlur(resized,(5,5),0)
            else:
                print("Chosen filter is not supported.")
                break
            
            _,ret_array = cv2.imencode('.jpg', filtered) #this fixed the pipeline issue as the imshow was converting to png.
            i = display.Image(data=ret_array)
            display.display(i)
            display.clear_output(wait=True)

    except KeyboardInterrupt:
        print("Released Video Resource from KeyboardInterrupt.")
    finally:
        del video

In [ ]:
showVideoWithFilter(5600, 0.5, 'canny') # or use 'blur' instead of 'canny'.

## Reference

[1] https://raymondlo84.medium.com/how-to-run-webcam-closer-to-real-time-performance-on-jupyter-notebook-python-3-in-2021-284c6c6364c6